### Question 1

### (a)

In [3]:
!pip install h2o

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 MB 4.3 MB/s eta 0:00:00


In [6]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
h2o.init()

airlines= h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/airlines/allyears2k_headers.zip")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.26" 2025-01-21; OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpjyq9hrww
  JVM stdout: /tmp/tmpjyq9hrww/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpjyq9hrww/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,1 month and 5 days
H2O_cluster_name:,H2O_from_python_unknownUser_jedjmn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.random_forest import H2ORandomForestEstimator

# splitting the dataset
train, test = airlines.split_frame(ratios=[.8], seed=1234)

# set the predictor names and the response column name
predictors = ["Origin", "Dest", "Year", "UniqueCarrier", "DayOfWeek", "Month", "Distance", "FlightNum"]
response = "IsDepDelayed"

hyperparams = {'ntrees': [10, 30, 50, 100], 'max_depth': [1, 2, 4, 6]}

rf_grid = H2OGridSearch(model=H2ORandomForestEstimator,
                        grid_id='rf_grid',
                        hyper_params=hyperparams)
rf_grid.train(x= predictors, y= response, training_frame=train)


drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


,max_depth,ntrees,model_ids,logloss
,6.0,50.0,rf_grid_model_12,0.6168946
,6.0,100.0,rf_grid_model_16,0.6177328
,6.0,30.0,rf_grid_model_8,0.6187844
,6.0,10.0,rf_grid_model_4,0.6238147
,4.0,30.0,rf_grid_model_7,0.6341642
,4.0,100.0,rf_grid_model_15,0.6348282
,4.0,50.0,rf_grid_model_11,0.6357348
,4.0,10.0,rf_grid_model_3,0.6423883
,2.0,30.0,rf_grid_model_6,0.6577542
,2.0,50.0,rf_grid_model_10,0.6582347


### (b)

In [8]:
rf_grid_results_sorted = rf_grid.get_grid(sort_by='accuracy', decreasing=True)
print(rf_grid_results_sorted)

Hyper-Parameter Search Summary: ordered by decreasing accuracy
    max_depth    ntrees    model_ids         accuracy
--  -----------  --------  ----------------  ----------
    6            50        rf_grid_model_12  0.670563
    6            100       rf_grid_model_16  0.670279
    6            30        rf_grid_model_8   0.667243
    6            10        rf_grid_model_4   0.661387
    4            100       rf_grid_model_15  0.655783
    4            30        rf_grid_model_7   0.654818
    4            50        rf_grid_model_11  0.653513
    4            10        rf_grid_model_3   0.638995
    2            100       rf_grid_model_14  0.636946
    2            50        rf_grid_model_10  0.632379
    2            30        rf_grid_model_6   0.626195
    1            100       rf_grid_model_13  0.619841
    2            10        rf_grid_model_2   0.616164
    1            50        rf_grid_model_9   0.611273
    1            30        rf_grid_model_5   0.60977
    1            1

### (c)

In [9]:
best_rf_model = rf_grid_results_sorted.models[0]
print("Best Model:\n", best_rf_model)

Best Model:
 Model Details
H2ORandomForestEstimator : Distributed Random Forest
Model Key: rf_grid_model_12


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 50                          63132                  6            6            6             40            64            58.82

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.21381304069775184
RMSE: 0.4623992222071225
LogLoss: 0.6168945526906364
Mean Per-Class Error: 0.37379313099714184
AUC: 0.723264098475117
AUCPR: 0.7351490081249785
Gini: 0.44652819695023394

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4024009231837267
       NO    YES    Error    Rate
-----  ----  -----  -------  -----------------
NO     66

In [10]:
best_model_performance = best_rf_model.model_performance(test)
print("AUC:\n", best_model_performance.auc())

AUC:
 0.7168688056882855


Based on the results from the grid search, we can see that the best model is the Distributed Random Forest model with 50 trees and a maximum depth of 6, and approximately 58.82 leaves on average (mean_leaves = 58.82). This model achieved an AUC score of approximately 0.717, indicating a moderate ability to distinguish between classes. Additionally, the model's MSE (Mean Squared Error) is about 0.214, suggesting a moderate level of error. Overall, this model demonstrates reasonably good performance in the given classification task.

### Question 2

### (a)

In [11]:
# using the same hyperparameters as before
hyperparams = {'ntrees': [10, 30, 50, 100], 'max_depth': [1, 2, 4, 6]}

# randomized grid search
search_config = {'strategy': 'RandomDiscrete', 'max_models': 10}
rf_random_grid_search = H2OGridSearch(model=H2ORandomForestEstimator,
                               hyper_params=hyperparams,
                               search_criteria=search_config,
                               grid_id='rf_random_grid')

rf_random_grid_search.train(x = predictors, y = response, training_frame=train)

drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


,max_depth,ntrees,model_ids,logloss
,6.0,30.0,rf_random_grid_model_5,0.6187762
,4.0,100.0,rf_random_grid_model_10,0.6341772
,4.0,50.0,rf_random_grid_model_8,0.6342086
,4.0,30.0,rf_random_grid_model_2,0.6349545
,4.0,10.0,rf_random_grid_model_1,0.6392263
,2.0,50.0,rf_random_grid_model_6,0.6584930
,2.0,10.0,rf_random_grid_model_7,0.6647663
,1.0,50.0,rf_random_grid_model_3,0.6730791
,1.0,30.0,rf_random_grid_model_9,0.6743547
,1.0,10.0,rf_random_grid_model_4,0.6752811


### (b)

In [12]:
rf_random_grid_results_sorted= rf_random_grid_search.get_grid(sort_by='accuracy', decreasing=True)
print(rf_random_grid_results_sorted)

Hyper-Parameter Search Summary: ordered by decreasing accuracy
    max_depth    ntrees    model_ids                accuracy
--  -----------  --------  -----------------------  ----------
    6            30        rf_random_grid_model_5   0.668577
    4            100       rf_random_grid_model_10  0.655442
    4            50        rf_random_grid_model_8   0.653513
    4            30        rf_random_grid_model_2   0.65218
    4            10        rf_random_grid_model_1   0.641053
    2            50        rf_random_grid_model_6   0.636691
    1            30        rf_random_grid_model_9   0.612011
    2            10        rf_random_grid_model_7   0.611907
    1            50        rf_random_grid_model_3   0.611359
    1            10        rf_random_grid_model_4   0.593106


### (c)

In [13]:
best_rf_random_results = rf_random_grid_results_sorted.models[0]
print("Best Model:\n", best_rf_random_results)

Best Model:
 Model Details
H2ORandomForestEstimator : Distributed Random Forest
Model Key: rf_random_grid_model_5


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    30                 30                          37128                  6            6            6             44            64            57.3

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.21462677310759906
RMSE: 0.4632782890527022
LogLoss: 0.6187761691024884
Mean Per-Class Error: 0.3929952247923985
AUC: 0.7208605550518584
AUCPR: 0.7310106981108566
Gini: 0.44172111010371684

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.38162467290942453
       NO    YES    Error    Rate
-----  ----  -----  -------  -----------------
NO 

In [14]:
best_randomized_model_performance = best_rf_random_results.model_performance(test)
print("AUC Score:", best_randomized_model_performance.auc())

AUC Score: 0.7193431322975012


Based on the results from the randomized grid search, we can see that the best model is the Distributed Random Forest model with 30 trees and a maximum depth of 6, and approximately 57.3 leaves on average (mean_leaves = 57.3). This model achieved an AUC score of approximately 0.719,


indicating a moderate ability to distinguish between classes. Additionally, the model's MSE (Mean Squared Error) is about 0.214, suggesting a moderate level of error. Overall, this model demonstrates reasonably good performance in the given classification task.


Based on the results, the optimal model identified by the randomized grid search has 30 trees, a maximum depth of 6, and an average of approximately 57.3 leaves. The AUC for this model is approximately 0.719, which is very similar to the result obtained from the traditional grid search (with a slight difference of 0.719 vs. 0.717). One advantage of using the randomized grid search is that it identifies the best model more quickly, and with fewer trees, this model is computationally less complex, which would result in reduced training time and less cost. While the performance was not compromised in this case, it's important to note that randomized grid search may not always guarantee optimal hyperparameter combinations, since it doesn't exhaustively search all possible combinations like the traditional grid search. Therefore, there is a risk that it might miss the best configuration if not carefully tuned.

### Question 3

### (a)

In [15]:
from h2o.automl import H2OAutoML

# initializing H2O AutoML
h20_automl = H2OAutoML(max_models=20, seed = 1234)

# running AutoML
h20_automl.train(x=predictors, y=response, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),9/20
# GBM base models (used / total),2/7
# XGBoost base models (used / total),4/6
# DRF base models (used / total),2/2
# DeepLearning base models (used / total),1/4
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


### (b)

In [16]:
# print leaderboard for all the different models
automl_leaderboard = h20_automl.leaderboard
print(automl_leaderboard)

model_id                                                     auc    logloss     aucpr    mean_per_class_error      rmse       mse
StackedEnsemble_AllModels_1_AutoML_1_20250503_13209     0.751336   0.589427  0.765226                0.34881   0.449602  0.202142
StackedEnsemble_BestOfFamily_1_AutoML_1_20250503_13209  0.750884   0.589894  0.76459                 0.357623  0.449805  0.202325
GBM_1_AutoML_1_20250503_13209                           0.74697    0.593642  0.760399                0.354801  0.451487  0.203841
XGBoost_grid_1_AutoML_1_20250503_13209_model_1          0.744813   0.595756  0.75885                 0.370461  0.452523  0.204777
XGBoost_grid_1_AutoML_1_20250503_13209_model_3          0.744372   0.595353  0.758751                0.367356  0.452442  0.204704
XRT_1_AutoML_1_20250503_13209                           0.744118   0.596984  0.755136                0.352605  0.452967  0.20518
XGBoost_1_AutoML_1_20250503_13209                       0.742837   0.597479  0.756966      

In [17]:
# identify the best performing model
best_model = h20_automl.leader
print("Best Model:", best_model.model_id)

Best Model: StackedEnsemble_AllModels_1_AutoML_1_20250503_13209


In [18]:
# printing the parameters of the best model
print("Model Parameters:")
print(best_model.params)

Model Parameters:
{'model_id': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': 'StackedEnsemble_AllModels_1_AutoML_1_20250503_13209', 'type': 'Key<Model>', 'URL': '/3/Models/StackedEnsemble_AllModels_1_AutoML_1_20250503_13209'}, 'input': None}, 'training_frame': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'FrameKeyV3', 'schema_type': 'Key<Frame>'}, 'name': 'AutoML_1_20250503_13209_training_py_2_sid_9275', 'type': 'Key<Frame>', 'URL': '/3/Frames/AutoML_1_20250503_13209_training_py_2_sid_9275'}, 'input': {'__meta': {'schema_version': 3, 'schema_name': 'FrameKeyV3', 'schema_type': 'Key<Frame>'}, 'name': 'AutoML_1_20250503_13209_training_py_2_sid_9275', 'type': 'Key<Frame>', 'URL': '/3/Frames/AutoML_1_20250503_13209_training_py_2_sid_9275'}}, 'response_column': {'default': None, 'actual': {'__meta': {'schema_version': 3, 'schema_name': 'ColSpecifierV3', 'schema_type': 'VecSpecifi

### (c)

In [19]:
# displaying AUC score of the best model
best_model_performance = best_model.model_performance(test)
print("AUC Score:", best_model_performance.auc())

AUC Score: 0.7542095320421767


### (d)

In [20]:
# finding the best XGBoost model with log loss as the criteria
best_xgboost_model = h20_automl.get_best_model(algorithm="xgboost", criterion="logloss")
print("Best XGBoost Model: ", best_xgboost_model.model_id)

xgboost_performance = best_xgboost_model.model_performance(test)
best_log_loss = xgboost_performance.logloss()
print("Best XGBoost Log Loss Model:", best_log_loss)

# finding the best model's AUC score
best_xgboost_performance = xgboost_performance.auc()
print("Best AUC Model:", best_xgboost_performance)

Best XGBoost Model:  XGBoost_grid_1_AutoML_1_20250503_13209_model_3
Best XGBoost Log Loss Model: 0.5935650477652386
Best AUC Model: 0.7448771492571172


Based on my results in the parts above, the best model identified by AutoML is a Stacked Ensemble model, specifically 'StackedEnsemble_AllModels_1_AutoML_1_20250503_13209'. This model achieved the highest performance across various metrics, including an AUC score of approximately 0.7542, indicating its strong ability to distinguish between classes.

<br>

The best XGBoost model, which was selected based on log loss, is 'XGBoost_grid_1_AutoML_1_20250503_13209_model_3', which achieved a log loss of approximately 0.5936 and an AUC of 0.7449.

<br>

The use of different models in the Stacked ensemble approach helps improve accuracy by combining various approaches. By looking at performance metrics like AUC, LogLoss, and RMSE, we are able to get a well-rounded picture of the model's strengths and weaknesses. To fully understand how the ensemble works, we need to consider how each individual model contributes to the overall predictions. While this can be a bit complex, it gives us valuable insights, especially for important decision-making.

<br>

Thus, the Stacked Ensemble model performs well on both training and cross-validation datasets, with solid results across AUC, LogLoss, and RMSE. Its complexity and range of metrics show that it is reliable and robust for this task. However, it takes longer to train. On the other hand, the XGBoost model is a bit faster to train, even though its accuracy is slightly lower. Since the difference in performance is minimal, the faster training time with XGBoost might make it a more practical choice in the real world.